### import

In [29]:
import torch
from torch import nn
from torch.functional import F
from torch.distributions import Multinomial, Categorical
from torch import optim
import numpy as np
import importlib as imp
import actor
import envs
import utils
imp.reload(actor)
imp.reload(envs)
imp.reload(utils)
from actor import Actor, Critic, LinearCritic
from envs import HospitalEnv
from utils import mlp_sample, ppo_update
from env_config import ENV_CONFIG as config

In [33]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 5池

#### 1.env actor初始化

In [30]:
state_dim = 2 * config["num_pools"] + 1
env = HospitalEnv(config)
act = Actor(config)
crtc = LinearCritic(state_dim)
state = env.reset()
state_tensor = state.float().unsqueeze(0)

In [23]:
state_tensor.shape

torch.Size([1, 11])

In [31]:
with torch.no_grad():
    logits = act(state_tensor).view(env.J, env.J)

In [32]:
env.step(logits)

(tensor([1, 0, 1, 3, 2, 0, 0, 0, 0, 0, 0]),
 tensor(0.),
 tensor([[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]], dtype=torch.int32))

In [33]:
traj = mlp_sample(env, act, config, is_random=True)

In [9]:
prob = F.softmax(logits, dim=-1)
prob, prob.shape

(tensor([[[3.6380e-20, 1.0587e-06, 1.0000e+00, 0.0000e+00, 5.0422e-14],
          [0.0000e+00, 2.1018e-19, 2.1018e-19, 2.1018e-19, 1.0000e+00],
          [0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00],
          [3.8123e-07, 1.5410e-05, 9.9998e-01, 3.8123e-07, 0.0000e+00],
          [4.6719e-27, 4.6719e-27, 1.0850e-22, 0.0000e+00, 1.0000e+00]]],
        grad_fn=<SoftmaxBackward0>),
 torch.Size([1, 5, 5]))

In [ ]:
env()

In [3]:
new_actor = Actor(J=J, B=B)
old_actor = Actor(J=J, B=B)

In [40]:
new_actor = Actor_GRU(J=J, B=B)
old_actor = Actor_GRU(J=J, B=B)

#### 3.训练

In [41]:
optimizer_actor = optim.Adam(new_actor.parameters(), lr=1e-3)
optimizer_crictic = optim.Adam(crictic.parameters(), lr=1e-3)
gamma=0.99
clip_ratio=0.2
train_epochs=5
batch_size=24